In [18]:
class Node:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
def generateNode(val):
    if not val:
        return None
    return Node(val)

## 1.二叉树按层遍历
队列完成,有左入左；有右入右

In [19]:
from collections import deque
#deque读取为O(N)，操作头尾为O(1)
#list读取为O(1)，操作头尾为O(N)
def BFS(head):
    if not head:
        return
    queue = deque()
    queue.append(head)
    while queue:
        cur = queue.popleft()
        print(cur.val, end=" ")
        if cur.left:
            queue.append(cur.left)
        if cur.right:
            queue.append(cur.right)
    return
    

## 2.二叉树的序列化和反序列化
* 序列化：结构转字符串存储
* 反序列化：由字符串转回二叉树结构
* 树和字符串一一对应，节点间打分隔符
* 先序、后序都可以作为序列化标准，但中序不可以

### 2.1 先序

In [20]:
# 先序 递归版 序列化
def pres(head, ans):
    if not head:
        ans.append(None) #字符串可以用特殊符号定义空子树
    else:
        ans.append(head.val)
        pres(head.left, ans)
        pres(head.right, ans)
    
def preSerial(head):
    ans = deque()
    pres(head, ans)
    return ans

# 先序 递归版 反序列化
def preb(preList):
    val = preList.popleft()
    if not val:
        return None
    head = generateNode(val)
    head.left = preb(preList)
    head.right = preb(preList)
    return head
    
def buildByPreSerial(preList):
    if not preList:
        return None
    return preb(preList)


### 2.2 后序

In [21]:
# 后序 递归版 序列化
def poss(head, ans):
    if not head:
        ans.append(None) #字符串可以用特殊符号定义空子树
    else:
        poss(head.left, ans)
        poss(head.right, ans)
        ans.append(head.val)
    
def posSerial(head):
    ans = deque()
    poss(head, ans)
    return ans

# 后序 递归版 反序列化
def posb(posList):
# 左右中 --> 中右左
    val = posList.pop()
    if not val:
        return None
    head = generateNode(val)
    head.right = preb(posList)
    head.left = preb(posList)
    return head
        
def buildByPosSerial(posList):
    if not posList:
        return None
#     stack = deque()
#     while posList:
#         stack.append(posList.popleft())
    return posb(posList)

### 2.3 按层

In [17]:
#序列化
def levelSerial(head):
    ans = deque()
    if not head:
        ans.append(None)
    else:
        ans.append(head.val)
        queue = deque()
        queue.append(head)
        while queue:
            head = queue.popleft() #父弹出的时候，序列化左右孩子。因为左右孩子可能为空！ 弹出再序列化会缺失
            if head.left:
                ans.append(head.left.val)
                queue.append(head.left)
            else:
                ans.append(None)
            if head.right:
                ans.append(head.right.val)
                queue.append(head.right)
            else:
                ans.append(None)
    return ans

#反序列化
def buildByLevelSerial(levelList):
    if not levelList:
        return None
    head = generateNode(levelList.popleft())
    queue = deque()
    if head:
        queue.append(head)
    while queue:
        node = queue.popleft()
        node.left = generateNode(levelList.popleft())
        node.right = generateNode(levelList.popleft())
        if not node.left:
            queue.append(node.left)
        if not node.right:
            queue.append(node.right)
    return head

### 2.4 多叉树转为二叉树

In [22]:
# 思路：把所有孩子，放在左树右边界上。右树完全不用。
class MultiNode:
    def __init__(self, val, children):
        self.val = val
        self.children = children #list
        
class TreeNode:
    def __init__(self, val, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        
def enc(children):
    head = None
    cur = None
    for child in children:
        tNode = TreeNode(child.val)
        if not head:
            head = tNode
        else:
            cur.right = tNode
        cur = tNode
        cur.left = enc(child.children) #深度优先遍历
    return head
        
def encode(root):
    if not root:
        return None
    head = TreeNode(root.val)
    head.left = enc(root.children)
    return head

def dec(root):
    children = list()
    while root:
        cur = TreeNode(root.val, dec(root.left))
        children.append(cur)
        root = root.right
    return children

def decode(root):
    if not root:
        return None
    return MultiNode(root.val, dec(root.left))
        

### 2.5 打印二叉树
https://github.com/algorithmzuo/algorithmbasic2020/blob/master/src/class11/Code04_PrintBinaryTree.java

In [1]:
def getSpace(num):
    space = ""
    for i in range(num):
        space+= " "
    return space
    

def printInOrder(head, height, to, length):
    if head is None:
        return 
    printInOrder(head.right, height+1, "↓", length)
    val = to + str(head.val) + to
    lenM= len(str(head.val))
    lenL = (length - lenM) // 2
    lenR = length - lenM - lenL
    val = getSpace(lenL) + val + getSpace(lenR)
    print(getSpace(height * length) + val)
    printInOrder(head.left, height+1, "↑", length)
def printBinaryTree(head):
    print("Binary Tree: ")
    printInOrder(head, 0, "H", 15)
    print()

### 2.6 求二叉树最宽的层
* 添加【当前层结束】、【下层结束】两个变量记录两个节点
* 当前层没有遇到【当前层结束】的节点时，每次从队列中弹出子孩子，赋值到【下层结束】，并且计数
* 当前层遇到【当前层结束】的节点时，从队列中弹出子孩子，赋值到【下层结束】，结束计数，更新最大层数
* 核心思路是当前层，就为下一层做准备，告诉下一层的截止位置。

In [24]:
# 简单版本用map记录层数对应节点数, 当前层时，把孩子记录map(孩子，curlevel+1)
# 优化版本，只用几个有限变量
def maxWidthNoMap(head):
    if not head:
        return 0
    queue = deque()
    queue.append(head)
    curEnd = head
    nextEnd = None
    max_width = 0
    curLevelNodes = 0
    while queue:
        cur = queue.popleft()
        if cur.left:
            queue.append(cur.left)
            nextEnd = cur.left
        if cur.right:
            queue.append(cur.right)
            nextEnd = cur.right
        curLevelNodes += 1
        if cur == curEnd:
            max_width = max(max_width, curLevelNodes)
            curLevelNodes = 0
            curEnd = nextEnd
    return max_width
    

### 2.7 返回中序序列上节点X的后继节点
* 题意Node定义增加一个父指针
* 某种(先中后序)序列化，查询x的后继节点
    1. 情况1：有右树，后继节点为右树左边界最左孩子
    2. 情况2：没有右树，找到x的父节点，如果此时x是父节点y的左孩子，停止，y即后继；否则继续往上找父节点。
             因为此时y的左孩子的右树刚刚完成，该完成y本身了（左中右顺序，左完了，该中了）
    3. 情况3：没有右树，没有找到y，本身节点为最后一个节点，没有后继节点

In [26]:
#题意给的Node类
class Node:
    def __init__(self, val, p=None, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        self.parent = p

In [28]:
def getLeftMost(node):
    if not node:
        return node
    while node.left:
        node = node.left
    return node

def getSuccessorNode(node):
    if not node:
        return node
    if node.right:
        return getLeftMost(node.right)
    else:
        parent = node.parent
        while parent and parent.right == node:
            node = parent
            parent = node.parent
        return parent

## 3.对折纸条
1. 对折一次：凹
2. 对折两次：凹凹凸
3. 对折三次：凹凹凸凹凹凸凸
       求对折N次的折痕
        

In [29]:
'''
思路：
一次对折:        1凹
二次对折:   2凹  1凹   2凸
三次对折:3凹2凹3凸1凹3凹2凸3凸

规律：每次对折，在中间折痕上下出现 凹、凸。 打印顺序即二叉树的中序遍历
'''

'\n思路：\n一次对折:        1凹\n二次对折:   2凹  1凹   2凸\n三次对折:3凹2凹3凸1凹3凹2凸3凸\n每次对折，在中间折痕上下出现 凹、凸\n'

In [30]:
#第一次凹
#所有左子树凹
#所有右子树凸

#来到一个节点，节点在i层，一共有N层，如果节点是凹convex=True，凸convex=False。
#函数功能，中序打印以这个节点为头的整棵树
def process(i, N, convex):
    if i > N:
        return 
    process(i+1, N, True)
    if convex:
        print("凹", end="")
    else:
        print("凸", end="")
    process(i+1, N, False)
    
def printAllFolds(N):
    process(1, N, True)